Reference: https://www.kaggle.com/kernels/scriptcontent/7827075/download

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
import os
import json
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import torch
is_torchvision_installed = True
try:
    import torchvision
except:
    is_torchvision_installed = False
torch.cuda.empty_cache()
import torch.utils.data
import random

class BalancedBatchSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset, labels=None):
        self.labels = labels
        self.dataset = dict()
        self.balanced_max = 0
        # Save all the indices for all the classes
        for idx in range(0, len(dataset)):
            label = dataset[idx][1]
            if label not in self.dataset:
                self.dataset[label] = list()
            self.dataset[label].append(idx)
            self.balanced_max = len(self.dataset[label]) \
                if len(self.dataset[label]) > self.balanced_max else self.balanced_max
        
        # Oversample the classes with fewer elements than the max
        for label in self.dataset:
            while len(self.dataset[label]) < self.balanced_max:
                self.dataset[label].append(random.choice(self.dataset[label]))
        self.keys = list(self.dataset.keys())
        self.currentkey = 0
        self.indices = [-1]*len(self.keys)

    def __iter__(self):
        while self.indices[self.currentkey] < self.balanced_max - 1:
            self.indices[self.currentkey] += 1
            yield self.dataset[self.keys[self.currentkey]][self.indices[self.currentkey]]
            self.currentkey = (self.currentkey + 1) % len(self.keys)
        self.indices = [-1]*len(self.keys)
    

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import autograd
import numpy as np
from torch.utils.data import DataLoader


class CramerRaoLowerBound:

    def __init__(self, model, crit, penalty, weight_decay_param, lr, weight=1000000):
        self.model = model
        self.weight = weight
        self.crit = crit
        if penalty in ('CRLBO'):
            self.optimizer = optim.Adam(self.model.parameters(), lr)
        elif penalty == 'SGD':
            self.optimizer = optim.SGD(self.model.parameters(), lr)
        elif penalty == 'L2':
            self.optimizer = optim.SGD(self.model.parameters(), lr, weight_decay=weight_decay_param)

    def forward_backward_update(self, input, target, penalty):
        output = self.model(input)
        if penalty in ('SGD', 'L2'):
            loss = self.crit(output, target)
        elif penalty == 'CRLBO':
            loss = self._compute_consolidation_loss(self.weight) + self.crit(output, target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def save(self, filename):
        torch.save(self.model, filename)

    def load(self, filename):
        self.model = torch.load(filename)
        
    def _update_mean_params(self):
        for param_name, param in self.model.named_parameters():
            _buff_param_name = param_name.replace('.', '__')
            self.model.register_buffer(_buff_param_name+'_estimated_mean', param.data.clone())

    def _update_fisher_params(self, current_ds, batch_size, num_batch):
        dl = DataLoader(current_ds, batch_size, shuffle=True)
        log_liklihoods = []
        for i, (input, target) in enumerate(dl):
            if i >= num_batch:
                break
            output = F.log_softmax(self.model(input.cuda().float()).detach().requires_grad_(True), dim=1)
            log_liklihoods.append(output[:, target])
        log_likelihood = torch.cat(log_liklihoods).mean()
        grad_log_liklihood = autograd.grad(log_likelihood, self.model.parameters(), allow_unused=True)
        _buff_param_names = [param[0].replace('.', '__') for param in self.model.named_parameters()]
        if(grad_log_liklihood[0] is not None):
            for _buff_param_name, param in zip(_buff_param_names, grad_log_liklihood):
                self.model.register_buffer(_buff_param_name+'_estimated_fisher', param.data.clone() ** 2)

    def register_crlbo_params(self, dataset, batch_size, num_batches):
        self._update_fisher_params(dataset, batch_size, num_batches)
        self._update_mean_params()

    def _compute_consolidation_loss(self, weight):
        try:
            losses = []
            for param_name, param in self.model.named_parameters():
                _buff_param_name = param_name.replace('.', '__')
                estimated_mean = getattr(self.model, '{}_estimated_mean'.format(_buff_param_name))
                estimated_fisher = getattr(self.model, '{}_estimated_fisher'.format(_buff_param_name))
                losses.append((estimated_fisher * (param - estimated_mean) ** 2).sum())
            return (weight / 2) * sum(losses)
        except AttributeError:
            return 0

Please visit the official website to find the dataset: https://www.cs.toronto.edu/~kriz/cifar.html

In [0]:
import torchvision
import torch

from torchvision import datasets, transforms

print('Loading data...')

# Image preprocessing, because VGG uses 224 * 224 size pictures, but CIFAR10 only has 32 * 32. In order to run the results faster,
# We enlarged them to 96 * 96 instead of 224 * 224 of the original paper
transform = transforms.Compose([
    transforms.Resize(96),# Scale to 96 * 96 size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True,
                                        transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

print('Loading is Done!')

Loading data...
Files already downloaded and verified
Files already downloaded and verified
Loading is Done!


In [0]:
print('Total Images: ', len(trainset))

Total Images:  50000


In [0]:
num_classes = len(trainset.classes)
print('No. of Classes: ', num_classes)

No. of Classes:  10


In [0]:
print('Classes: ', trainset.class_to_idx)

Classes:  {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [0]:
class_sample_counts = np.bincount(trainset.targets)
print(class_sample_counts)

[5000 5000 5000 5000 5000 5000 5000 5000 5000 5000]


### Viewing the datasets

In [0]:
# fig = plt.figure(figsize=(14,10))

# for n in range(1, 10):
#   fig.add_subplot(4, 7, n)
#   img = trainset.data[n]
#   plt.imshow(img)
#   plt.title(trainset.classes[trainset.targets[n]])
#   plt.axis('off')

In [0]:
len(trainset)

50000

In [0]:
batches = 5000

train_batch_size_ = int(len(trainset) / batches)
print('Train Batch size: ', train_batch_size_)

Train Batch size:  10


In [0]:
len(testset)

batches = 5


test_batch_size_ = int(len(testset) / batches)
print('Test Batch size', test_batch_size_)

Test Batch size 2000


In [0]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_size_,sampler=BalancedBatchSampler(trainset, list(trainset.class_to_idx.values())), shuffle=False, num_workers = 4)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size_, sampler=BalancedBatchSampler(testset, list(testset.class_to_idx.values())), shuffle=False, num_workers = 4)


## Load and Compile the Model

In [0]:
from torch.autograd import Variable

# Hyperparameter
EPOCH = 100
learning_rate = 0.000001

# Whether to use GPU
use_gpu = True


class VGG(nn.Module):
    '''
    VGG model
    '''
    def __init__(self, conv_features):
        super(VGG, self).__init__()
        self.conv_features = conv_features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(73728, 512), 
            nn.ReLU(True),
            nn.Dropout(),
            # nn.Linear(512, 512),
            # nn.ReLU(True),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.conv_features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Build the conv layer of the loop
def make_layers(struct, in_channels=1, batch_norm=False):
    layers = []
    for out_channels in struct:
        if out_channels == 'pooling':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = out_channels
    return nn.Sequential(*layers)


In [0]:
crit = nn.CrossEntropyLoss()

In [0]:
# Model initialization
vgg_conv_layers = [64, 64, 'pooling', 
                   128, 128, 'pooling'
                  #  , 
                  #  256, 256, 256, 'pooling', 
                  #  512, 512, 512, 'pooling', 
                  #  512, 512, 512, 'pooling'
                   ]

# Initial channel-three channels
vgg16 = VGG(make_layers(vgg_conv_layers, in_channels=3))



In [0]:
# Whether to use GPU
if use_gpu:
    vgg16 = vgg16.cuda()

penalty_ = 'CRLBO' #Penalty = 'SGD', 'CRLBO', 'L2'
crlbo = CramerRaoLowerBound(vgg16, crit=crit, penalty = penalty_, weight_decay_param = 0.1, lr=learning_rate, weight=1000000)
print('Model Specs: ', crlbo.model.train())

Model Specs:  VGG(
  (conv_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=73728, out_features=512, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Training the model

In [0]:
import warnings
warnings.filterwarnings("ignore")

model_files_path = r'\content\drive\My Drive\Colab Notebooks\Models'
# crlbo.save(model_files_path + '\\' + penalty_ + '_batch_' + str(10011) + '.h5')

In [0]:
from tqdm import tqdm

i = 0

for b, (images, labels) in enumerate(tqdm(train_loader)):

    if use_gpu:
        images = images.cuda()
        labels = labels.cuda()
        
    for epoch in range(EPOCH):
        total = 0
        correct = 0

        crlbo.forward_backward_update(images, labels, penalty_)

        # End an iteration        
        if epoch == EPOCH-1:
            # Calculate training accuracy
            y_pred = crlbo.model(images)
            _, predicted = torch.max(y_pred.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.data).sum()
            print('Accuracy of the model on the train images: %d %%' % (100 * correct / total), 'Epoch : ', epoch )
    
    i = i + 1

    if penalty_ == 'CRLBO':
      crlbo.register_crlbo_params(trainset, 10, batches)
    

1it [00:01,  1.91s/it]

Accuracy of the model on the train images: 60 % Epoch :  99


2it [00:03,  1.87s/it]

Accuracy of the model on the train images: 90 % Epoch :  99


3it [00:05,  1.84s/it]

...

Accuracy of the model on the train images: 90 % Epoch :  99


722it [21:25,  1.78s/it]

Accuracy of the model on the train images: 80 % Epoch :  99


723it [21:26,  1.78s/it]

Accuracy of the model on the train images: 100 % Epoch :  99


In [0]:
j = 0
result = []
torch.cuda.empty_cache()
with torch.no_grad():
    for test_images, test_labels in tqdm(test_loader):
        print('\n' , 'Test Batch -  ' , j, '--', np.bincount(test_labels), 'Test Batch size: ' , test_images.shape[0])
      
        # Test
        model = crlbo.model.eval()
        model = model.cuda()

        correct = 0
        total = 0

        if use_gpu:
            test_images = test_images.cuda()
            test_labels = test_labels.cuda()
        y_pred = model(test_images.cuda().float()).detach()
        _, predicted = torch.max(y_pred.data, 1)
        total += test_labels.size(0)
        temp = (predicted == test_labels.data).sum()
        correct += temp

        acc = (100 * correct / total)

        print('\n' , 'Test Batch ', j, ' : Accuracy: %d %%' % acc)
        result.append(acc)
        j = j + 1


In [0]:
pd.DataFrame(data = result)

In [0]:
pd.DataFrame(data = result).to_csv(penalty_ + '_results.csv')